In [1]:
import tensorflow as tf
import math
from tensorflow.examples.tutorials.mnist import input_data as mnist_data

In [2]:
print("Tensorflow version " + tf.__version__)
tf.set_random_seed(0)

Tensorflow version 1.6.0


In [3]:
mnist = mnist_data.read_data_sets("data",
                                  one_hot=True,
                                  reshape=False,
                                  validation_size=0)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [4]:
# input X: 28x28 grayscale images,
# the first dimension (None) will index
# the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])

# correct answers will go here
Y_ = tf.placeholder(tf.float32, [None, 10])

N = 100 # Batch Size

In [5]:
D1 = 200 
D2 = 250
D3 = 250
D4 = 100

W1 = tf.Variable(tf.truncated_normal([28 * 28, D1], stddev=0.1))
B1 = tf.Variable(tf.zeros([D1]))

W2 = tf.Variable(tf.truncated_normal([D1, D2], stddev=0.1))
B2 = tf.Variable(tf.zeros([D2]))

W3 = tf.Variable(tf.truncated_normal([D2, D3], stddev=0.1))
B3 = tf.Variable(tf.zeros([D3]))

W4 = tf.Variable(tf.truncated_normal([D3, D4], stddev=0.1))
B4 = tf.Variable(tf.zeros([D4]))

W5 = tf.Variable(tf.truncated_normal([D4, 10], stddev=0.1))
B5 = tf.Variable(tf.zeros([10]))

In [6]:
H0 = tf.reshape(X, [-1, 28 * 28])

"""
H1 = tf.nn.sigmoid(tf.matmul(H0, W1) + B1)
H2 = tf.nn.sigmoid(tf.matmul(H1, W2) + B2)
H3 = tf.nn.sigmoid(tf.matmul(H2, W3) + B3)
H4 = tf.nn.sigmoid(tf.matmul(H3, W4) + B4)
"""

pdrop = tf.placeholder(tf.float32)

H1 = tf.nn.relu(tf.matmul(H0, W1) + B1)
H1d = tf.nn.dropout(H1, pdrop)

H2 = tf.nn.relu(tf.matmul(H1d, W2) + B2)
H2d = tf.nn.dropout(H2, pdrop)

H3 = tf.nn.relu(tf.matmul(H2d, W3) + B3)
H3d = tf.nn.dropout(H3, pdrop)

H4 = tf.nn.relu(tf.matmul(H3d, W4) + B4)
H4d = tf.nn.dropout(H4, pdrop)

H5 = tf.matmul(H4d, W5) + B5
Y = tf.nn.softmax(H5)

In [7]:
# *10 since we really want sum of loss and not mean
# cross_entropy = -tf.reduce_mean(Y_ * tf.log(Y)) * 10.0

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=H5, labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy) * N

correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# training, learning rate = 0.005
# You always forget to optimize!
# Optimizer begin backprop process

alpha = 0.005

# Vanilla Gradient Descent
# train_step = tf.train.GradientDescentOptimizer(alpha).minimize(cross_entropy)

step = tf.placeholder(tf.int32)

"""
tf.train.exponential_decay(
    learning_rate,
    global_step,
    decay_steps,
    decay_rate,
    staircase=False,
    name=None
)

decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)
"""

lr = 0.0001 + tf.train.exponential_decay(alpha, step, 2000, 1/math.e)

# Adam Optimizer
train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

In [9]:
# init
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [10]:
iterations = 5000
train_data_log_freq = 500

In [12]:
for i in range(iterations):
    # training on batches of 100 images with N labels
    batch_X, batch_Y = mnist.train.next_batch(N)
    train_data = {
        X: batch_X,
        Y_: batch_Y,
        step: i,
        pdrop: 0.75
    }
    
    """
    I was stuck coz sess.run([accuracy, cross_entropy] 
    was before sess.run(train_step
    why would the order matter here?
    since train_step includes cross_entropy as a dependecy
    """
    
    sess.run(train_step, feed_dict = train_data)
    
    train_acc, train_loss = sess.run([accuracy, cross_entropy],
                    feed_dict = train_data)

    test_data = {
        X: mnist.test.images,
        Y_: mnist.test.labels,
        pdrop: 1.00
    }
    test_acc, test_loss = sess.run([accuracy, cross_entropy], feed_dict = test_data)
    
    if i % train_data_log_freq == 0:
        print("Train: {:>5d} | Acc: {:>5.4f} | Loss: {:>8.4f}".format(i, train_acc, train_loss))
        print("Test:: {:>5d} | Acc: {:>5.4f} | Loss: {:>8.4f}".format(i, test_acc, test_loss))
        print("")
    

Train:     0 | Acc: 0.1900 | Loss: 216.5357
Test::     0 | Acc: 0.2598 | Loss: 215.7285

Train:   500 | Acc: 0.9500 | Loss:  22.3320
Test::   500 | Acc: 0.9530 | Loss:  18.0577

Train:  1000 | Acc: 0.9900 | Loss:   6.1557
Test::  1000 | Acc: 0.9607 | Loss:  13.4446

Train:  1500 | Acc: 0.9600 | Loss:  21.2021
Test::  1500 | Acc: 0.9684 | Loss:  10.9393

Train:  2000 | Acc: 0.9800 | Loss:  13.1411
Test::  2000 | Acc: 0.9723 | Loss:  10.0044

Train:  2500 | Acc: 0.9800 | Loss:   7.5530
Test::  2500 | Acc: 0.9741 | Loss:   9.6001

Train:  3000 | Acc: 0.9900 | Loss:   5.8472
Test::  3000 | Acc: 0.9764 | Loss:   8.9709

Train:  3500 | Acc: 0.9800 | Loss:   6.1212
Test::  3500 | Acc: 0.9766 | Loss:   8.9820

Train:  4000 | Acc: 0.9500 | Loss:  14.9557
Test::  4000 | Acc: 0.9787 | Loss:   8.1218

Train:  4500 | Acc: 0.9800 | Loss:   5.1555
Test::  4500 | Acc: 0.9789 | Loss:   7.8454

